# [完整代码](#full-code)

# Import Libraries

In [54]:
import backtrader as bt
import pandas as pd
import numpy as np
# from get_score import get_score_ch
from sqlalchemy import create_engine
import talib as ta
import datetime
import pandas as pd

In [55]:
engine= create_engine('sqlite:///../../dataset/us/us_ticker_price_yf.db')

# Gather Data

In [56]:
df = pd.read_sql('INTC', engine)
df.rename(columns={'hgih':'high'},inplace=True)
# df['chg_pct'] = ((df.close - df.open)/df.open)*100
df['close'] = df['close'].apply(lambda x: round(x,3))
df

,open,high,low,close,Adj Close,volume,datetime,VMA20
0,0.325521,0.330729,0.325521,0.326,0.192692,10924800,1980-03-17,NaN
1,0.325521,0.328125,0.322917,0.323,0.191150,17068800,1980-03-18,NaN
2,0.330729,0.335938,0.330729,0.331,0.195774,18508800,1980-03-19,NaN
3,0.330729,0.334635,0.329427,0.329,0.195004,11174400,1980-03-20,NaN
4,0.322917,0.322917,0.317708,0.318,0.188067,12172800,1980-03-21,NaN
...,...,...,...,...,...,...,...,...
10668,37.660000,38.139999,37.389999,37.990,37.990002,22554800,2022-07-08,36854415.0
10669,37.660000,37.669998,37.029999,37.200,37.200001,25904300,2022-07-11,36197270.0
10670,37.650002,37.700001,36.990002,37.210,37.209999,30546300,2022-07-12,35571275.0
10671,36.680000,37.430000,36.619999,37.210,37.209999,27059200,2022-07-13,34407440.0


# Main Function

## Set Time

In [57]:
# 140约等于200天
df = df[9834:9984]
df.index = range(len(df))
df_length = len(df)

## Gather Data

In [59]:
df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
# 判断是不是10天内的极值点
df['extremum'] = 0
for i in range(5,df_length):
    # or df.close.loc[i] == df.close.loc[i-5:i+5].max() 
    if df.close.loc[i] == df.close.loc[i-5:i+5].min():
        df['extremum'].loc[i] = 1
        
minimum_index = np.array(df[df['extremum']==1].index)
minimum_value = np.array(df[df['extremum']==1].close)

In [60]:
# find the min1
min1_val = np.min(minimum_value)
min1_pos = np.argmin(minimum_value)
min1_ind = minimum_index[min1_pos]
# find the max1
max1_val = df.iloc[:min1_ind].close.max()

# min2
min2_val = np.min(minimum_value[min1_pos+1:])
min2_pos = np.argmin(minimum_value[min1_pos+1:]) + min1_pos + 1
min2_ind = minimum_index[min2_pos]
# max2
max2_val = df.iloc[min1_ind:min2_ind].close.max()

# min3
min3_val = np.min(minimum_value[min2_pos+1:])
min3_pos = np.argmin(minimum_value[min2_pos+1:]) + min2_pos + 1
min3_ind = minimum_index[min3_pos]
# max3
max3_val = df.iloc[min2_ind:min3_ind].close.max()

## Select Condition

### prepare data

In [65]:
vol1 = df.iloc[min1_ind].VMA20
vol2 = df.iloc[min2_ind].VMA20
vol3 = df.iloc[min3_ind].VMA20

In [66]:
vol1, vol2, vol3

(30011880.0, 25265195.0, 19399655.0)

In [67]:
dec1 = (max1_val - min1_val) / max1_val
dec2 = (max2_val - min2_val) / max2_val
dec3 = (max3_val - min3_val) / max3_val

In [68]:
dec1,dec2,dec3

(0.2611356681400884, 0.15041572184429328, 0.06828900207508012)

In [70]:
# 
if vol1 > vol2 > vol3 and dec1 > dec2 >dec3 and max2_val*1.05 > max3_val:
    print('ticker is vcp')
else:
    print('not vcp pattern')

ticker is vcp


<h1><span id="full-code">Full code</span></h1>

In [ ]:
# 获取Volume MA
df['VMA20'] = ta.SMA(df.volume, timeperiod=20)
# 判断是不是10天内的极值点
df['extremum'] = 0
for i in range(5,df_length):
    # or df.close.loc[i] == df.close.loc[i-5:i+5].max() 
    if df.close.loc[i] == df.close.loc[i-5:i+5].min():
        df['extremum'].loc[i] = 1
        
minimum_index = np.array(df[df['extremum']==1].index)
minimum_value = np.array(df[df['extremum']==1].close)

# Main
# 140约等于200天
df = df[9834:9984]
df.index = range(len(df))
df_length = len(df)

# find the min1
min1_val = np.min(minimum_value)
min1_pos = np.argmin(minimum_value)
min1_ind = minimum_index[min1_pos]
# find the max1
max1_val = df.iloc[:min1_ind].close.max()

# min2
min2_val = np.min(minimum_value[min1_pos+1:])
min2_pos = np.argmin(minimum_value[min1_pos+1:]) + min1_pos + 1
min2_ind = minimum_index[min2_pos]
# max2
max2_val = df.iloc[min1_ind:min2_ind].close.max()

# min3
min3_val = np.min(minimum_value[min2_pos+1:])
min3_pos = np.argmin(minimum_value[min2_pos+1:]) + min2_pos + 1
min3_ind = minimum_index[min3_pos]
# max3
max3_val = df.iloc[min2_ind:min3_ind].close.max()

vol1 = df.iloc[min1_ind].VMA20
vol2 = df.iloc[min2_ind].VMA20
vol3 = df.iloc[min3_ind].VMA20

dec1 = (max1_val - min1_val) / max1_val
dec2 = (max2_val - min2_val) / max2_val
dec3 = (max3_val - min3_val) / max3_val

# 
if vol1 > vol2 > vol3 and dec1 > dec2 >dec3 and max2_val*1.05 > max3_val:
    print('ticker is vcp')
else:
    print('not vcp pattern')